# 2. 데이터 정제 – 이상 데이터 처리

### 이상치 다루기
1. 이상치 확인
    - Z-Score
    - IQR (Interquartile Range)
2. 이상치 처리
    - 삭제
    - 대체<br><br>
        
### 실습 내용 요약
- 예제 데이터 내 이상치 확인 및 삭제, 대체 방안 실습

In [1]:
import numpy as np
import pandas as pd

In [3]:
# 데이터 로딩 및 개요 확인
click_data = pd.read_csv("./data/click.csv")
click_data.head(15)

,category,journal,num_click
0,eng,D,164
1,tra,B,762
2,eng,D,220
3,tra,B,639
4,spo,A,894
5,soc,E,348
6,pol,C,521
7,spo,E,563
8,fin,B,250
9,pol,A,620


In [4]:
# 데이터 개요 파악
click_data.info()

# 데이터 copy
click_copy = click_data.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17734 entries, 0 to 17733
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   category   17734 non-null  object
 1   journal    17734 non-null  object
 2   num_click  17734 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 415.8+ KB


In [5]:
# describe 함수 활용, 수치형 변수인 num_click 컬럼의 통계 확인
click_copy['num_click'].describe()

count     17734.000000
mean        543.845495
std        5619.881420
min           0.000000
25%         325.000000
50%         459.000000
75%         597.000000
max      433992.000000
Name: num_click, dtype: float64

## 2-1. 이상치 확인 및 삭제하기

### 1) Z-score 방식 & 삭제

In [7]:
# Z-score 컬럼 생성
click_copy['z_score'] = (click_copy['num_click'] - np.mean(click_copy['num_click'])) / np.std(click_copy['num_click'])
click_copy.head()

,category,journal,num_click,z-score
0,eng,D,164,-0.067591
1,tra,B,762,0.038819
2,eng,D,220,-0.057627
3,tra,B,639,0.016932
4,spo,A,894,0.062308


In [8]:
# 원본 데이터 내 z-score 확인
# 일반적으로 -3 아래, 3 위 범위를 이상치로 상정한다
click_copy.describe()

,num_click,z-score
count,17734.000000,1.773400e+04
mean,543.845495,1.247154e-17
std,5619.881420,1.000028e+00
min,0.000000,-9.677443e-02
25%,325.000000,-3.894240e-02
50%,459.000000,-1.509781e-02
75%,597.000000,9.458563e-03
max,433992.000000,7.712981e+01


In [10]:
# 이상치 처리하기 (삭제)
# z-score 기반 이상치 제거 후 데이터 차원 확인
click_copy = click_copy[(click_copy['z-score'] < 3) & (click_copy['z-score'] > -3)]

# 데이터 개요 확인, 4건의 데이터가 삭제됨
click_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17730 entries, 0 to 17733
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   category   17730 non-null  object 
 1   journal    17730 non-null  object 
 2   num_click  17730 non-null  int64  
 3   z-score    17730 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 692.6+ KB


In [11]:
# 이상치 제거 후 데이터 요약 통계 확인
# z-score 의 min, max 가 절대값 3을 초과하지 않음을 확인
click_copy.describe()

,num_click,z-score
count,17730.000000,17730.000000
mean,460.268697,-0.014872
std,199.068617,0.035423
min,0.000000,-0.096774
25%,325.000000,-0.038942
50%,459.000000,-0.015098
75%,597.000000,0.009459
max,1295.000000,0.133664


In [13]:
# Scipy 라이브러리에서 z-score 메소드 제공
from scipy.stats import zscore
click_copy = click_data.copy()

click_copy['z_score'] = zscore(click_copy['num_click'])
click_copy = click_copy[(click_copy['z_score'] < 3) & (click_copy['z_score'] > -3)]
print(click_copy.shape) 

(17730, 4)


In [14]:
# 기존 결과와 비교
click_copy.describe()

,num_click,z_score
count,17730.000000,17730.000000
mean,460.268697,-0.014872
std,199.068617,0.035423
min,0.000000,-0.096774
25%,325.000000,-0.038942
50%,459.000000,-0.015098
75%,597.000000,0.009459
max,1295.000000,0.133664


### 2) IQR 방식 & 삭제

In [16]:
# IQR 기반 이상치 처리
# 원 데이터 copy
click_copy = click_data.copy()

In [17]:
# 1, 3 분위수 (Q1, Q3) 구하기
q1 = click_copy['num_click'].quantile(0.25)
q3 = click_copy['num_click'].quantile(0.75)

# IQR 구하기 (Interquartile Range)
iqr = q3 - q1
iqr

272.0

In [18]:
# IQR 기반 이상치 제거하기
click_copy = click_copy[(click_copy['num_click'] > (q1 - 1.5*iqr)) & (click_copy['num_click'] < (q3 + 1.5*iqr))]
click_copy.describe()

,num_click
count,17682.000000
mean,458.620235
std,196.779622
min,0.000000
25%,325.000000
50%,459.000000
75%,595.000000
max,1003.000000


In [22]:
click_copy

,category,journal,num_click
0,eng,D,164
1,tra,B,762
2,eng,D,220
3,tra,B,639
4,spo,A,894
...,...,...,...
17729,spo,E,456
17730,fin,D,433992
17731,fin,D,310
17732,eng,A,540


In [19]:
# 이상치 제거 후 데이터 확인
print(np.shape(click_copy))

(17682, 3)


## 2-2. 이상치 대체하기

In [20]:
# 이상치 대체
click_copy = click_data.copy()

In [21]:
# 특정 조건이 명시되어 있어서 이상치를 해당 조건으로 대체한다는 시나리오
max_click = 1000

In [23]:
# 정의된 기준으로 대체 후 비교를 위해 컬럼 복사
click_copy['new_num_click'] = click_copy['num_click']

# 이상치 대체
# 내가 인덱싱 이해 못해서 남기는 해석
# loc(행, 렬) 인덱싱인데, clock_copyp['new_num_click'] > max_click 을 만족하는 행들의
# 'new_num_click' 열을 max_click으로 바꾸라는 의미
click_copy.loc[click_copy['new_num_click'] > max_click, 'new_num_click'] = max_click
click_copy.describe()

,num_click,new_num_click
count,17734.000000,17734.000000
mean,543.845495,460.207229
std,5619.881420,198.658304
min,0.000000,0.000000
25%,325.000000,325.000000
50%,459.000000,459.000000
75%,597.000000,597.000000
max,433992.000000,1000.000000


In [24]:
click_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17734 entries, 0 to 17733
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   category       17734 non-null  object
 1   journal        17734 non-null  object
 2   num_click      17734 non-null  int64 
 3   new_num_click  17734 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 554.3+ KB
